In [17]:
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader

import random
import numpy as np
import time

from dataset import MonuSegDataSet, MonuSegValDataSet
from auxilary.utils import readConfig

from networkModules.probUNet.probabilistic_unet import ProbabilisticUnet
from networkModules.probUNet.utils import l2_regularisation

In [18]:
# Set seed
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
np.random.seed(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)
torch.set_grad_enabled(True)
start_time = time.time()

In [19]:
# Read configFile
config = readConfig('config.sys')

In [20]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [21]:
print('configuring data loaders')
train_dataset = MonuSegDataSet(config["trainDataset"], config)
train_data = DataLoader(train_dataset,batch_size=config["batch_size"],shuffle=True)
val_dataset = MonuSegValDataSet(config["valDataset"], config)
val_data = DataLoader(val_dataset,batch_size=1,num_workers=4)

configuring data loaders


In [22]:
no_channels = 3
no_classes = 2
filter_list = [32, 64, 128, 128]
latent_dim = 2
no_fcomb_convs = 4
beta = 10.0

epochs = 10

In [23]:
net = ProbabilisticUnet(no_channels,no_classes,filter_list,latent_dim,no_fcomb_convs,beta)
net.to(device)
optimizer = torch.optim.Adam(net.parameters(), lr=1e-4, weight_decay=0)
for epoch in range(epochs):
    for step, (patch, mask) in enumerate(train_data): 
        patch = patch.to(device)
        mask = mask.to(device)
        mask = torch.unsqueeze(mask,1)
        net.forward(patch, mask, training=True)
        elbo = net.elbo(mask)
        reg_loss = l2_regularisation(net.posterior) + l2_regularisation(net.prior) + l2_regularisation(net.fcomb.layers)
        loss = -elbo + 1e-5 * reg_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

RuntimeError: Tensors must have same number of dimensions: got 4 and 5